## Database statistics for SCI Data publication - Will give the contents of Table 1

In [1]:
from SlideRunner.dataAccess.database import Database
import SlideRunner.dataAccess.database
import os
import numpy as np
import openslide
import cv2

In [2]:

test_slides = ['14','18','3','22','10','15','21']



## Query all databases for the respective classes. Joint query to have all in the same order.

In [3]:
DB = Database().open('MITOS_WSI_CMC_ODAEL_TR.sqlite')
DB.execute('ATTACH `MITOS_WSI_CMC_MEL.sqlite` as MEL;')
DB.execute('ATTACH `MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite` as CODAEL;')

ODAEL = DB.execute("""
SELECT * FROM (
SELECT filename, MitoticFigures, MitoticFigureLookalikes FROM 
(SELECT COUNT(*) as MitoticFigureLookalikes, slide from Annotations where agreedClass==1 group by slide) as cls1
LEFT JOIN (SELECT COUNT(*) as MitoticFigures, slide from Annotations where agreedClass==2 group by slide) as cls2 on cls2.slide == cls1.slide
LEFT JOIN Slides on Slides.uid == cls2.slide order by mitoticFigures) as ODAEL
LEFT JOIN 
(
SELECT filename, MitoticFigures, MitoticFigureLookalikes FROM (
SELECT COUNT(*) as MitoticFigures, slide from MEL.Annotations where agreedClass==2 group by slide) as cls2
LEFT JOIN (SELECT COUNT(*) as MitoticFigureLookalikes, slide from MEL.Annotations where agreedClass==1 group by slide) as cls1 on cls1.slide == cls2.slide
LEFT JOIN MEL.Slides on Slides.uid == cls2.slide order by mitoticFigures
) as MEL on MEL.filename == ODAEL.filename
LEFT JOIN 
(
SELECT filename, MitoticFigures, MitoticFigureLookalikes FROM 
(SELECT COUNT(*) as MitoticFigureLookalikes, slide from CODAEL.Annotations where agreedClass==1 group by slide) as cls1
LEFT JOIN (SELECT COUNT(*) as MitoticFigures, slide from CODAEL.Annotations where agreedClass==2 group by slide) as cls2 on cls2.slide == cls1.slide
LEFT JOIN MEL.Slides on Slides.uid == cls2.slide order by mitoticFigures
) as CODAEL on CODAEL.filename == ODAEL.filename order by CODAEL.mitoticFigures


""").fetchall()

Output everything in LateX table format

In [4]:
DB.execute("""SELECT COUNT(*) as MitoticFigureLookalikes, slide from Annotations where agreedClass==7 group by slide""").fetchall()

[]

In [5]:
ODAEL[0]

('4eee7b944ad5e46c60ce.svs',
 61,
 196,
 '4eee7b944ad5e46c60ce.svs',
 47,
 114,
 '4eee7b944ad5e46c60ce.svs',
 64,
 193)

In [6]:


dbTumor = Database()

dbTumor.open('./MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite')
totalarea = 0
totalmit = 0
totalnonmit = 0
totalmit_mel, totalnonmit_mel = 0,0
totalmit_odael, totalnonmit_odael = 0,0
for idx, (filename, mitA, nonmitA, _, mit,nonmit, _,  mitC,nonmitC) in enumerate(ODAEL):

    dbTSlide = dbTumor.findSlideWithFilename(os.path.basename(filename), slidepath='')
    dbTumor.loadIntoMemory(dbTSlide)
    
    slide = openslide.open_slide('../WSI/'+filename)


    positiveId = dbTumor.findClassidOfClass('Tumor region')[0][0]
    import matplotlib.path as p

    micronsPerPixel = slide.properties[openslide.PROPERTY_NAME_MPP_X]
    mpp_scaled = float(slide.level_downsamples[-1]) * float(micronsPerPixel)  

    slideActiveMap = np.zeros((slide.level_dimensions[-1][::-1]))
    ds = slide.level_downsamples[-1]
    for anno in dbTumor.annotations.keys():
        if (dbTumor.annotations[anno].agreedLabel()==positiveId) and (type(dbTumor.annotations[anno]) == type(dbTumor.annotations[anno]) == SlideRunner.dataAccess.annotations.polygonAnnotation):
            co = dbTumor.annotations[anno].coordinates
            p3 = np.int32([np.asarray(co)/ds])
            cv2.fillPoly(slideActiveMap, p3,color=[255] )

    for anno in dbTumor.annotations.keys():
        if not (dbTumor.annotations[anno].agreedLabel()==positiveId) and (type(dbTumor.annotations[anno]) == type(dbTumor.annotations[anno]) == SlideRunner.dataAccess.annotations.polygonAnnotation):
            co = dbTumor.annotations[anno].coordinates
            p3 = np.int32([np.asarray(co)/ds])
            cv2.fillPoly(slideActiveMap, p3,color=[0] )
    mm2 = np.sum(slideActiveMap>0)*mpp_scaled*mpp_scaled/1E6
    totalarea += mm2
    totalmit += mitC
    totalnonmit += nonmitC
    
    totalmit_mel, totalnonmit_mel = totalmit_mel+mit, totalnonmit_mel+nonmit
    totalmit_odael, totalnonmit_odael = totalmit_odael+mitA, totalnonmit_odael+nonmitA
    
    fname = filename.replace('_','\\_')
    print(f"{idx+1} & {fname} & {mm2:.2f}\,$\\mathrm"+"{mm2}"+f"^2$ & {mit:,d} / {mitA:,d} / {mitC:,d} & {nonmit:,d} / {nonmitA:,d} / {nonmitC:,d} & {'test' if str(dbTSlide) in test_slides else 'train'} \\\\ ")
    

1 & 4eee7b944ad5e46c60ce.svs & 66.06\,$\mathrm{mm2}^2$ & 47 / 61 / 64 & 114 / 196 / 193 & test \\ 
2 & a8773be388e12df89edd.svs & 37.01\,$\mathrm{mm2}^2$ & 64 / 71 / 74 & 204 / 591 / 588 & train \\ 
3 & deb768e5efb9d1dcbc13.svs & 187.43\,$\mathrm{mm2}^2$ & 92 / 96 / 84 & 287 / 472 / 484 & train \\ 
4 & e09512d530d933e436d5.svs & 214.97\,$\mathrm{mm2}^2$ & 87 / 98 / 102 & 602 / 742 / 738 & test \\ 
5 & 72c93e042d0171a61012.svs & 26.29\,$\mathrm{mm2}^2$ & 130 / 151 / 140 & 375 / 680 / 691 & train \\ 
6 & 2d56d1902ca533a5b509.svs & 49.32\,$\mathrm{mm2}^2$ & 139 / 155 / 153 & 228 / 365 / 367 & test \\ 
7 & 084383c18b9060880e82.svs & 41.71\,$\mathrm{mm2}^2$ & 157 / 173 / 160 & 404 / 547 / 560 & train \\ 
8 & da18e7b9846e9d38034c.svs & 253.10\,$\mathrm{mm2}^2$ & 187 / 210 / 211 & 991 / 1,354 / 1,353 & train \\ 
9 & 13528f1921d4f1f15511.svs & 339.93\,$\mathrm{mm2}^2$ & 283 / 301 / 292 & 963 / 1,127 / 1,136 & test \\ 
10 & d0423ef9a648bb66a763.svs & 273.88\,$\mathrm{mm2}^2$ & 378 / 411 / 354 &

In [7]:
totalnonmit_mel, totalnonmit_odael, totalnonmit

(26526, 36135, 36379)

In [8]:
totalmit_mel, totalmit_odael, totalmit


(13342, 14151, 13907)

## And finally the question: What is the total area?

In [9]:
print('Total area: ',totalarea,'mm^2')
print('mitotic figures in CODAEL data set',totalmit)
print('nonmitotic figures in CODAEL data set',totalnonmit)


Total area:  4360.07788915561 mm^2
mitotic figures in CODAEL data set 13907
nonmitotic figures in CODAEL data set 36379
